<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/BanglaSER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!gdown --id 1fZoWnL0neesTfj3l1kQRNN5O0Sfm3Bjl
time.sleep(5)

!unzip /content/BanglaSER.zip

In [7]:
import os
import librosa
import json
import glob
from sklearn.model_selection import train_test_split

# Emotion mapping based on your dataset description
emotion_map = {
    1: "Happy",
    2: "Sad",
    3: "Angry",
    4: "Surprise",
    5: "Neutral"
}

def parse_filename(filename):
    """Parses the filename to extract the emotion label."""
    parts = filename.split('-')
    if len(parts) >= 3:  # Adjust based on your filename structure
        emotion_label = int(parts[2])  # Assumes the 3rd part is the emotion label
        return emotion_map.get(emotion_label, "Unknown")
    return None

def calculate_audio_length(file_path):
    """Calculates the length of an audio file in seconds."""
    audio, sr = librosa.load(file_path, sr=None)
    return librosa.get_duration(y=audio, sr=sr)

def create_filtered_data(directory):
    """Creates a minimal JSON data structure for audio files in a directory, excluding 'Surprise' emotion."""
    json_data = []

    # Using glob to find all WAV files in the directory
    audio_files = glob.glob(os.path.join(directory, '**/*.wav'), recursive=True)

    for file_path in audio_files:
        filename = os.path.basename(file_path)
        audio_id = os.path.splitext(filename)[0]
        emotion_label = parse_filename(filename)

        # Exclude files with emotion "Surprise"
        if emotion_label and emotion_label != "Surprise":
            audio_length = calculate_audio_length(file_path)

            # Create minimal data entry
            data_entry = {
                "id": audio_id,
                "emotion": emotion_label,
                "length": audio_length,
                "wav": file_path
            }
            json_data.append(data_entry)

    return json_data

def split_and_save_data(json_data, seed=32):
    """Splits the data into train and test sets and saves them to separate JSON files."""
    train, test = train_test_split(json_data, test_size=0.2, random_state=seed)

    # Save train data
    with open('train_data.json', 'w') as train_file:
        json.dump(train, train_file, indent=4)

    # Save test data
    with open('test_data.json', 'w') as test_file:
        json.dump(test, test_file, indent=4)



In [8]:
# Directory path for your audio files
directory_path = '/content/t9h6p943xy-5/BanglaSER'

# Step 1: Create filtered data directly
print("Creating filtered data...")
filtered_data = create_filtered_data(directory_path)

# Step 2: Split and save data to train and test JSON files
print("Splitting and saving data into train and test JSON files...")
split_and_save_data(filtered_data)

print("Train and test JSON files created successfully.")


Creating filtered data...
Splitting and saving data into train and test JSON files...
Train and test JSON files created successfully.


In [10]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/t9h6p943xy-5/BanglaSER/train_data.json",'r') as f:
    train_data = json.load(f)

with open("/content/t9h6p943xy-5/BanglaSER/test_data.json",'r') as f:
    test_data = json.load(f)

In [11]:

train_data

[{'id': '03-01-01-01-01-01-16',
  'emotion': 'Happy',
  'length': 3.8286041666666666,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 16/03-01-01-01-01-01-16.wav'},
 {'id': '03-01-02-01-03-02-01',
  'emotion': 'Sad',
  'length': 3.5936875,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 01/03-01-02-01-03-02-01.wav'},
 {'id': '03-01-01-01-03-01-30',
  'emotion': 'Happy',
  'length': 3.2443083900226757,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 30/03-01-01-01-03-01-30.wav'},
 {'id': '03-01-05-01-03-03-17',
  'emotion': 'Neutral',
  'length': 3.03843537414966,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 17/03-01-05-01-03-03-17.wav'},
 {'id': '03-01-02-01-01-03-02',
  'emotion': 'Sad',
  'length': 3.3687755102040815,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 02/03-01-02-01-01-03-02.wav'},
 {'id': '03-01-01-01-02-03-02',
  'emotion': 'Happy',
  'length': 3.794875283446712,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 02/03-01-01-01-02-03-02.wav'},
 {'id': '03-01-01-01-02-

In [12]:
test_data

[{'id': '03-01-05-01-02-01-05',
  'emotion': 'Neutral',
  'length': 3.918548752834467,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 05/03-01-05-01-02-01-05.wav'},
 {'id': '03-01-05-01-02-01-04',
  'emotion': 'Neutral',
  'length': 3.5210657596371884,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 04/03-01-05-01-02-01-04.wav'},
 {'id': '03-01-01-01-03-01-08',
  'emotion': 'Happy',
  'length': 3.42975,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 08/03-01-01-01-03-01-08.wav'},
 {'id': '03-01-03-02-01-02-23',
  'emotion': 'Angry',
  'length': 3.0677777777777777,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 23/03-01-03-02-01-02-23.wav'},
 {'id': '03-01-05-01-02-03-04',
  'emotion': 'Neutral',
  'length': 3.273106575963719,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 04/03-01-05-01-02-03-04.wav'},
 {'id': '03-01-01-01-03-01-24',
  'emotion': 'Happy',
  'length': 3.8630625,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 24/03-01-01-01-03-01-24.wav'},
 {'id': '03-01-05-01-03-03

In [13]:

train_samples_list = [
    {**sample_data, "id": i} for i, sample_data in enumerate(train_data)
]
train_dataset = Dataset.from_list(train_samples_list)

In [14]:
test_samples_list = [
    {**sample_data, "id": i} for i, sample_data in enumerate(test_data)
]
test_dataset = Dataset.from_list(test_samples_list)

In [15]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 928
    })
    test: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 233
    })
    dev: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 233
    })
})

In [16]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [17]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'],
        num_rows: 928
    })
    test: Dataset({
        features: ['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'],
        num_rows: 233
    })
    dev: Dataset({
        features: ['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'],
        num_rows: 233
    })
})

In [18]:

dataset['train'][0].keys()

dict_keys(['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'])

In [19]:
audio_data = dataset['train'][0]['audio_array'] # Access data using the 'dataset' variable
print(audio_data)

[9.401794613950187e-07, -3.829346042039106e-06, -6.776192549295956e-07, -6.454843628489471e-07, 1.717127815936692e-06, -6.9281977630453184e-06, -1.1851529961859342e-06, -1.6177846191567369e-06, -9.88875171969994e-07, 2.447838141961256e-06, -6.7834935180144385e-06, 2.022654371103272e-06, -6.945020686544012e-06, 2.187196969316574e-06, -5.232650437392294e-06, -5.34774790139636e-06, 4.3988911784254014e-06, -7.964374162838794e-06, 4.12007011618698e-07, -6.44348438072484e-06, -1.8826390260073822e-06, -8.783301268522337e-07, -5.253450581221841e-06, -1.7961195908355876e-06, -8.914148565963842e-06, -5.091248453936714e-07, -8.856755812303163e-06, -4.144467311562039e-06, -7.891302630014252e-06, -5.166712071513757e-06, -2.4968603611341678e-06, -7.055305559333647e-06, -8.599068905823515e-07, -7.863931386964396e-06, -6.054823643353302e-06, -2.618120106490096e-06, -5.691641945304582e-06, -4.593178346112836e-06, -9.558173132973025e-07, -4.419491688167909e-06, -2.0987572497688234e-06, -4.73366253572749

In [20]:

!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter
!pip install tqdm
from tqdm import tqdm


In [21]:
from sklearn.metrics import recall_score, accuracy_score
import torch
from torch.nn.utils.rnn import pad_sequence

# ---- Metric Functions ----
def calculate_unweighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='macro')

def calculate_weighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='weighted')

def calculate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

# ---- Collate Function for DataLoader ----
def collate_fn(batch):
    # Sort the batch by input length in descending order
    batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)

    # Extract inputs and labels
    inputs = [torch.tensor(example["input_values"]) for example in batch]
    emotions = [int(example["emotion"]) for example in batch]  # Replace "label" with "emotion"

    # Pad sequences
    inputs = pad_sequence(inputs, batch_first=True)

    return {"input_values": inputs, "emotion": torch.tensor(emotions)}



In [22]:
from datasets import Dataset, DatasetDict, Audio
import pandas as pd
import glob
import json

# Function to check if an emotion is common
def is_common(emotion):
    return emotion in ['Angry', 'Happy', 'Neutral', 'Sad']

# Function to convert emotion to numerical ID
def to_label(emotion, label2id):
    return label2id[emotion]

# Function to load and process BanglaSER dataset
def banglaser_data(session=None):
    all_files = glob.glob("/content/t9h6p943xy-5/BanglaSER/*")
    banglaser_df = pd.DataFrame()

    for file in all_files:
        # Check if the item is a file before opening
        if os.path.isfile(file):
            with open(file, "r") as f:
                data = json.load(f)

                # Check if data is a list and convert if necessary
                if isinstance(data, list):
                    data = {i: item for i, item in enumerate(data)}  # Create a dictionary with index as keys

                df = pd.DataFrame.from_dict(data, orient="index").reset_index()
                df["set"] = [file.split("/")[-1].split(".")[0] for _ in range(len(df))]
                df = df.rename(columns={'wav': 'audio', 'emotion': 'emotion'})
                df["emotion_flag"] = df["emotion"].apply(is_common)
                banglaser_df = pd.concat([banglaser_df, df], axis=0)

    # Filter common emotions
    banglaser_df = banglaser_df.loc[banglaser_df["emotion_flag"] == 1].reset_index(drop=True)

    # Ensure 'set' column has the correct values before filtering
    print("Unique values in 'set' column:", banglaser_df['set'].unique())  # Print unique values for debugging

    banglaser_df_train = banglaser_df.loc[banglaser_df["set"] != "test"][["audio", "emotion"]]
    banglaser_df_test = banglaser_df.loc[banglaser_df["set"] == "test"][["audio", "emotion"]]

    # If test set is empty, use a portion of the train set for validation
    if banglaser_df_test.empty:
        print("Test set is empty. Using a portion of the train set for validation.")
        from sklearn.model_selection import train_test_split
        banglaser_df_train, banglaser_df_test = train_test_split(banglaser_df_train, test_size=0.2, random_state=42)

    banglaser_df_dev = banglaser_df_test.copy() # Use test as dev


    # Check if DataFrames are empty and handle accordingly
    if banglaser_df_train.empty:
        raise ValueError("Training DataFrame is empty. Check data loading or filtering.")
    if banglaser_df_test.empty: # This should not happen now, but kept for safety
        raise ValueError("Test DataFrame is empty. Check data loading or filtering.")

    emotions = ['Angry', 'Happy', 'Neutral', 'Sad']
    label2id, id2label = {emotion: str(i) for i, emotion in enumerate(emotions)}, {str(i): emotion for i, emotion in enumerate(emotions)}

    # Map emotions to IDs
    train_df, test_df = banglaser_df_train, banglaser_df_test
    train_df["emotion"] = train_df["emotion"].apply(to_label, args=(label2id,)).values
    test_df["emotion"] = test_df["emotion"].apply(to_label, args=(label2id,)).values

    # Create Hugging Face datasets with audio casting
    train_audio_data = Dataset.from_pandas(train_df[['audio', 'emotion']]).cast_column("audio", Audio(sampling_rate=16_000))
    test_audio_data = Dataset.from_pandas(test_df[['audio', 'emotion']]).cast_column("audio", Audio(sampling_rate=16_000))

    # Create a DatasetDict with train, test, and dev splits (dev will be test split here since it's identical)
    return label2id, id2label, DatasetDict({'train': train_audio_data, 'test': test_audio_data, 'dev': test_audio_data})

# Preprocessing function for feature extraction
def preprocess_function(examples, feature_extractor):
    max_duration = 20.0  # Maximum duration of audio in seconds
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        padding=True,
        truncation=True
    )
    return inputs


In [23]:
import os
import torch
from torch.utils.tensorboard import SummaryWriter
from transformers import AutoModelForAudioClassification
from sklearn.metrics import recall_score
import numpy as np

# Define the model function
def model_ft(label2id, id2label, num_classes, device="cpu"):
    model_name = "facebook/wav2vec2-base"
    model = AutoModelForAudioClassification.from_pretrained(
        model_name,
        num_labels=num_classes,
        label2id=label2id,
        id2label=id2label
    )
    model.to(device)
    return model

# Define the training pipeline function
def pipeline_ft(model, train_loader, valid_loader, test_loader, hyperparam, device):
    SESSION = hyperparam['SESSION']
    LEARNING_RATE = hyperparam['LEARNING_RATE']
    TRAINING = hyperparam['TRAINING']
    N_EPOCHS = hyperparam['N_EPOCHS']
    LINGUALITY = hyperparam['LINGUALITY']
    LANGUAGE = hyperparam['LANGUAGE']

    file_path = f"/content/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}.pth"
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Load checkpoint if exists
    epoch = 1
    if os.path.exists(file_path):
        checkpoint = torch.load(file_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        total_train_loss = checkpoint.get('training loss', 0)
        total_valid_loss = checkpoint.get('validation loss', 0)
        print("Model checkpoint has been loaded")

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir=f"/content/tensorboards/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}")

    # Evaluation-only mode
    if TRAINING == 0:
        test_unweighted_recall, test_weighted_recall, test_accuracy = validation(
            model,
            tqdm(test_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Test", leave=False),
            loss_fn,
            device
        )
        print(f"Epoch {epoch}/{N_EPOCHS}, Test Recall (unweighted): {test_unweighted_recall}, Recall (weighted): {test_weighted_recall}, Accuracy: {test_accuracy}\n")

    else:
        # Training and validation mode
        for epoch in range(epoch, N_EPOCHS+1):
            # Train step - adjust to returned values
            train_unweighted_recall, train_weighted_recall, train_accuracy = train(
                model,
                tqdm(train_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Training", leave=False),
                optimizer,
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Training Recall (unweighted): {train_unweighted_recall}, Recall (weighted): {train_weighted_recall}, Accuracy: {train_accuracy}")

            # Log training metrics
            writer.add_scalar("Accuracy/Train", train_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Train", train_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Train", train_weighted_recall, epoch)

            # Validation step - adjust to returned values
            valid_unweighted_recall, valid_weighted_recall, valid_accuracy = validation(
                model,
                tqdm(valid_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Validation", leave=False),
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Validation Recall (unweighted): {valid_unweighted_recall}, Recall (weighted): {valid_weighted_recall}, Accuracy: {valid_accuracy}\n")

            # Log validation metrics
            writer.add_scalar("Accuracy/Validation", valid_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Validation", valid_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Validation", valid_weighted_recall, epoch)

            writer.flush()

            # Save model checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'training loss': train_unweighted_recall,  # Adjust as needed
                'validation loss': valid_unweighted_recall  # Adjust as needed
            }, file_path)

            print(f"Model has been saved after epoch: {epoch}\n")

        # Close TensorBoard writer
        writer.close()

# Train function
def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_train_loss, correct_predictions, total_predictions = 0, 0, 0
    all_actual_labels, all_predictions = [], []

    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['emotion'].to(device)  # Adjusted 'label' to 'emotion'
        optimizer.zero_grad()
        outputs = model(inputs).logits
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    return unweighted_recall, correct_predictions / total_predictions, total_train_loss / len(train_loader)

# Validation function
def validation(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch['input_values'].to(device), batch['emotion'].to(device)  # Adjusted 'label' to 'emotion'
            outputs = model(inputs).logits
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            predictions = torch.argmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    unweighted_recall = recall_score(all_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_labels, all_predictions, average='weighted')
    accuracy = sum(np.array(all_labels) == np.array(all_predictions)) / len(all_labels)

    return unweighted_recall, weighted_recall, accuracy


In [24]:
class Args:
    def __init__(self):
        self.N_EPOCHS = 10
        self.LEARNING_RATE = 1e-5
        self.BATCH_SIZE = 8
        self.SESSION = 1
        self.TRAINING = 1
        self.PARADIGM = "FT"
        self.LANGUAGE = "FR"
        self.LINGUALITY = "Monolingual"

# Example usage:
args = Args()
print(args.N_EPOCHS)  # Output: 10
print(args.LANGUAGE)  # Output: "FR"


10
FR


In [25]:
import random
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import Wav2Vec2FeatureExtractor
from datasets import DatasetDict

def main(args_dict):
    # Set random seed for reproducibility
    random_seed = 42
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_seed)

    # Access values from args_dict
    session = args_dict["SESSION"]
    batch_size = args_dict["BATCH_SIZE"]
    learning_rate = args_dict["LEARNING_RATE"]

    # Load BanglaSER dataset using your banglaser_data function
    label2id, id2label, ds = banglaser_data(session=session)  # Your function to load BanglaSER

    # Initialize feature extractor for Wav2Vec2 model
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")

    # Preprocess the dataset
    encoded_audio = ds.map(
        lambda example: preprocess_function(example, feature_extractor),
        remove_columns=["audio"],
        batched=True
    )

    # Initialize DataLoaders for train, test, and validation splits
    train_loader = DataLoader(encoded_audio["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(encoded_audio["test"], batch_size=batch_size, collate_fn=collate_fn)
    valid_loader = DataLoader(encoded_audio["test"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # Model initialization
    model = model_ft(label2id, id2label, len(label2id), device="cuda" if torch.cuda.is_available() else "cpu")
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    # Run the fine-tuning pipeline
    pipeline_ft(model, train_loader, valid_loader, test_loader, args_dict, device="cuda" if torch.cuda.is_available() else "cpu")



In [26]:

# Create an instance of Args and pass it as a dictionary to main
args = Args()
main(args.__dict__)

Unique values in 'set' column: ['train_data' 'test_data']
Test set is empty. Using a portion of the train set for validation.


Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

...

Epoch 8/10, Validation Recall (unweighted): 0.910529519172246, Recall (weighted): 0.9098712446351931, Accuracy: 0.9098712446351931

Model has been saved after epoch: 8



Epoch 9/10, Training Recall (unweighted): 0.9768136746696345, Recall (weighted): 0.9762931034482759, Accuracy: 0.11243457546264961


Epoch 9/10, Validation Recall (unweighted): 0.9286366402921485, Recall (weighted): 0.9313304721030042, Accuracy: 0.9313304721030042

Model has been saved after epoch: 9



Epoch 10/10, Training Recall (unweighted): 0.9823790025956601, Recall (weighted): 0.9816810344827587, Accuracy: 0.08208741421072648


Epoch 10/10, Validation Recall (unweighted): 0.8630037605425616, Recall (weighted): 0.8626609442060086, Accuracy: 0.8626609442060086

Model has been saved after epoch: 10

